In [192]:
import requests 
import base64
import datetime
from urllib.parse import urlencode
import pandas as pd
import json

In [193]:
class SpotifyAPI(object):
    access_token = None
    client_id = None
    user_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, user_id):
        self.client_id = client_id
        self.client_secret = client_secret
        self.user_id = user_id
    
    def get_client_credentials(self):
        client_id = self.client_id
        client_secret = self.client_secret
        
        if client_id == None or client_secret == None:
            raise Exception("client_id or client_secret needs to be completed")
            
        client_credentials = f"{client_id}:{client_secret}"
        client_credentials_b64 = base64.b64encode(client_credentials.encode())
        return client_credentials_b64.decode()
    
    def get_token_header(self):
        return {"Authorization":f"Basic {self.get_client_credentials()}"}
        
    def get_token_data(self):
        return {"grant_type":"client_credentials"}
        
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_header()
        

        r = requests.post(token_url, data = token_data, headers = token_headers)
        
        data = r.json()
        now = datetime.datetime.now()
        self.access_token = data["access_token"]
        
    def search(self, track_name, search_type = "artist"):
        endpoint = "https://api.spotify.com/v1/search"
        header = self.get_resource_header()
        data = urlencode({"q":track_name,
               "type":search_type.lower()})
        lookup_url = f"{endpoint}?{data}"

        r = requests.get(lookup_url, headers = header)
        if r.status_code in range(200,299):
            return r.json()
        return {}
    

        
    def get_resource_header(self):
        header = {
            "Authorization":f"Bearer {self.access_token}"
        }
        return header
    
    def get_resource(self, lookup_id, resource_type = 'albums'):
        endpoint = f"https://api.spotify.com/v1/{resource_type}/{lookup_id}"
        header = self.get_resource_header()
        r = requests.get(endpoint,headers = header)
        return r.json()
    
    def get_artist(self, _id):
        return self.get_resource(_id,resource_type = "artists")
    
    def get_album(self, _id):
        return self.get_resource(_id,resource_type = "albums")
    
    def new_playlist(self,user_id):
        pass
        
        

In [194]:
#enter the client id, client secret, and spotify ID
client_id = "f2ace4e1b01a4385af648c12a8b0879e"
client_secret = "1d0aa652e0e34680a4c13ee7d033402f"
user_id = "3hoqlfah25ej3gr7e4x3vsf4c"

spotify = SpotifyAPI(client_id,client_secret, user_id)


In [195]:
spotify.perform_auth()

In [196]:
endpoint = f"https://api.spotify.com/v1/users/{user_id}/playlists"
token = "BQD3ji2oUQUIB3Heor6-BbJ3aP8Mr2alPaoblnIJnc-sx7AgB9VWlRDHnUHxkDaFIg2kzsFB4ZhGxwhLCfn5b_Og0dCM6S7iS2ElvwWyacMavJVK6Wh1bjVYl0N1pQvfY0kVIlm8w1-wdJclO1_1aymhwPo9EutSjQnMcc6kufKOB79aePZXUrrDllZbE63lFfPnEsGdCfHJyWwVT5DDIKCRJzb17HrZ"
header = {
            "Authorization":"Bearer {}".format(token),
            "Content-Type":"application/json"
        }
r = requests.post(endpoint,data = json.dumps({"name":"hello"}), headers = header)

In [197]:
r.json()

{'collaborative': False,
 'description': None,
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/41BdszaR4w7OFs7v5npUVM'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/41BdszaR4w7OFs7v5npUVM',
 'id': '41BdszaR4w7OFs7v5npUVM',
 'images': [],
 'name': 'hello',
 'owner': {'display_name': 'Kevin',
  'external_urls': {'spotify': 'https://open.spotify.com/user/3hoqlfah25ej3gr7e4x3vsf4c'},
  'href': 'https://api.spotify.com/v1/users/3hoqlfah25ej3gr7e4x3vsf4c',
  'id': '3hoqlfah25ej3gr7e4x3vsf4c',
  'type': 'user',
  'uri': 'spotify:user:3hoqlfah25ej3gr7e4x3vsf4c'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MSwyOGJjMjE0MWFiNTRiMDI1YzlkMjJkY2NjMDE1ZTFmMWIwNDhjNTFh',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/41BdszaR4w7OFs7v5npUVM/tracks',
  'items': [],
  'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0},
 'type': 'playlist',
 'uri': 'spotify:playlist:41BdszaR4w7OFs7v5npUVM'}

In [158]:
endpoint = "https://accounts.spotify.com/authorize"
data = urlencode({"client_id":spotify.client_id,
               "response_type":"code",
                 "redirect_uri":'https://example.com/callback'})
lookup_url = f"{endpoint}?{data}"
r = requests.get(lookup_url)
r.url

'https://accounts.spotify.com/login?continue=https%3A%2F%2Faccounts.spotify.com%2Fauthorize%3Fclient_id%3Df2ace4e1b01a4385af648c12a8b0879e%26response_type%3Dcode%26redirect_uri%3Dhttps%253A%252F%252Fexample.com%252Fcallback'

In [162]:
r.text

'\n<!DOCTYPE html>\n<html id="app" lang="en" dir="ltr" ng-csp ng-strict-di>\n<head>\n  <meta charset="utf-8">\n  <title ng-bind="(&#39;loginTitle&#39; | localize) + &#39; - Spotify&#39;">Spotify</title>\n  <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">\n  <base href="/">\n  <link rel="icon" href="https://accounts.scdn.co/images/favicon.ace4d8543bbb017893402a1e9d1ac1fa.ico">\n  <link href="https://accounts.scdn.co/css/index.eeaa6d4523dd0f6a9559.css" media="screen" rel="stylesheet">\n\n  <script defer src="https://accounts.scdn.co/js/index.eeaa6d4523dd0f6a9559.js" sp-bootstrap></script>\n  <meta ng-non-bindable sp-bootstrap-data=\'{"phoneFeatureEnabled":false,"previewEnabled":false,"user":false,"tpaState":"AQDdLb03edVpeq4QphKClityD1fOwieS5xR6y/16Hotagfn9XeovK9eELhQuwoozP4eWvnTtVb3U2JqG7E9ELYLWuyj3I5csKKicUV3iFJOVvI9PjzHP/ttvNrpLMbcJiOTiQDq7RSIUULV+eOOXP5Jcc1gm+oQxQfGw3x3DDvKwYIKGchFdZnvrX73R/rJBISpc50h/RXauimjb/z/TWeLitUaVsRRyF